<a href="https://colab.research.google.com/github/Geethussanthosh/Cosmic_object_identifier/blob/main/cosmic_object.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import warnings
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16

In [3]:
plt.rc('figure', autolayout = True)
plt.rc('axes', labelweight = 'bold', titleweight = 'bold', titlesize = 18, titlepad = 10, labelsize = 'large')
plt.rc('image', cmap = 'magma')
warnings.filterwarnings("ignore")

In [4]:
def seed_set(seed = 31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC-OPS'] = '1'
seed_set(31415)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
train_dir = '/content/drive/MyDrive/train/train'
valid_dir = '/content/drive/MyDrive/valid/valid'

print("Training directory:",train_dir)
print("Validation directory:",valid_dir)

ds_train_ = image_dataset_from_directory(
    train_dir,
    labels = 'inferred',
    label_mode = 'categorical',
    image_size = [128, 128],
    interpolation = 'nearest',
    batch_size = 64,
    shuffle = True,
)

ds_valid_ = image_dataset_from_directory(
    train_dir,
    labels = 'inferred',
    label_mode = 'categorical',
    image_size = [128, 128],
    interpolation = 'nearest',
    batch_size = 64,
    shuffle = False,
)


Training directory: /content/drive/MyDrive/train/train
Validation directory: /content/drive/MyDrive/valid/valid
Found 368 files belonging to 8 classes.
Found 368 files belonging to 8 classes.


In [8]:
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype = tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = (
    ds_train_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size = AUTOTUNE)
)
ds_valid = (
    ds_valid_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size = AUTOTUNE)
)

In [9]:
pretrained_base = VGG16(weights = 'imagenet',include_top = False, input_shape = (128,128,3))

pretrained_base.trainable = False

58889256/58889256 [==============================] - 3s 0us/step


In [10]:
model = keras.Sequential([
    pretrained_base,
    layers.Flatten(),
    layers.Dense(256, activation = 'relu'),
    layers.Dropout(0.5),
    layers.Dense(8, activation = 'softmax')
])
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)


history = model.fit(
    ds_train,validation_data = ds_valid,
    epochs = 30,
    verbose = 1
)

Epoch 1/30
6/6 [==============================] - 272s 27s/step - loss: 1.5854 - accuracy: 0.4755 - val_loss: 0.2696 - val_accuracy: 1.0000
Epoch 2/30
6/6 [==============================] - 128s 24s/step - loss: 0.3636 - accuracy: 0.8696 - val_loss: 0.1177 - val_accuracy: 1.0000
Epoch 3/30
6/6 [==============================] - 138s 25s/step - loss: 0.1994 - accuracy: 0.9185 - val_loss: 0.0495 - val_accuracy: 1.0000
Epoch 4/30
6/6 [==============================] - 131s 24s/step - loss: 0.1161 - accuracy: 0.9647 - val_loss: 0.0310 - val_accuracy: 1.0000
Epoch 5/30
6/6 [==============================] - 129s 24s/step - loss: 0.0739 - accuracy: 0.9864 - val_loss: 0.0175 - val_accuracy: 1.0000
Epoch 6/30
6/6 [==============================] - 134s 24s/step - loss: 0.0462 - accuracy: 0.9946 - val_loss: 0.0092 - val_accuracy: 1.0000
Epoch 7/30
6/6 [==============================] - 137s 25s/step - loss: 0.0334 - accuracy: 0.9973 - val_loss: 0.0059 - val_accuracy: 1.0000
Epoch 8/30
6/6 [====

In [11]:
labels = ['Earth', 'Jupiter', 'Mars', 'Mercury', 'Neptune', 'Saturn', 'Uranus', 'Venus']

In [12]:
#preprocess image
def preprocess_img(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels = 3)
    image = tf.image.resize(image, [128,128])
    image = tf.cast(image, tf.float32) / 255.0
    return image

In [13]:
def predict_class(image_path):
    image = preprocess_img(image_path)
    image = tf.expand_dims(image, axis = 0)
    predictions = model.predict(image)

    predicted_index = tf.argmax(predictions[0]).numpy()
    predicted_class = labels[predicted_index]
    return predicted_class

In [17]:
#test
image_path = '/content/drive/MyDrive/planets/Neptune1.jpg'
planet = predict_class(image_path)
print("The predicted planet is: ", planet)




1/1 [==============================] - 0s 172ms/step
The predicted planet is:  Neptune
